# This file is largely copied
see: https://www.kaggle.com/fabiendaniel/hyperparameter-tuning/notebook

We do need to change to make it our own.

# Team 3 - Merchant Category Recommendation
## Hyperparameter Tuning

### Team 3
- Vinicio De Sola
- Kevin Hanna
- Pri Nonis
- Bradley Nott


Here we add engineer new features and write the files out to:
    - input/engineered_train.pkl
    - input/engineered_test.pkl

In [1]:
import numpy               as np
import matplotlib.pyplot   as plt
import pandas              as pd
import seaborn             as sb

from datetime              import timedelta, datetime

from sklearn               import metrics
from sklearn.decomposition import PCA
from sklearn.cluster       import KMeans
from sklearn.mixture       import GaussianMixture
from sklearn.linear_model  import LinearRegression

from matplotlib.colors     import LogNorm

from IPython.display       import HTML, Markdown

from sklearn.metrics       import mean_squared_error

%matplotlib inline

np.random.seed(0)

import os
print(os.listdir("./input"))

['historical_transactions.csv', '.ipynb_checkpoints', 'new_merchant_transactions.csv', 'engineered_test.pkl', '.~lock.Data_Dictionary.xlsx#', 'merchants.csv', 'sample_submission.csv', 'test.csv', 'Data_Dictionary.xlsx', 'engineered_train.pkl', '.gitignore', 'train.csv']


## Read in all the data files

In [2]:
train        = pd.read_pickle('input/engineered_train.pkl')
test         = pd.read_pickle('input/engineered_test.pkl')

target = train['target']
del train['target']

## A view of our training data dataframe

In [3]:
display(train.head())
display(test.head())
display(target.head())

,feature_3,first_active_delta_days,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,...,new_category_2_5.0_sum,new_category_3_A_sum,new_category_3_B_sum,new_category_3_C_sum,new_authorized_flag_y_sum,new_authorized_flag_n_sum,new_month_diff_mean,new_month_diff_min,new_month_diff_max,new_month_diff_std
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,1,246,0,0,0,0,1,0,1,0,...,0.0,23.0,0.0,0.0,23.0,0.0,13.521739,13.0,14.0,0.510754
C_ID_3d0044924f,0,397,0,0,0,1,0,1,0,0,...,0.0,0.0,6.0,0.0,6.0,0.0,14.500000,14.0,15.0,0.547723
C_ID_d639edf6cd,0,550,0,1,0,0,0,0,1,0,...,1.0,1.0,0.0,0.0,1.0,0.0,13.000000,13.0,13.0,0.000000
C_ID_186d6a6901,0,154,0,0,0,1,0,0,0,1,...,0.0,0.0,6.0,0.0,7.0,0.0,13.571429,13.0,14.0,0.534522
C_ID_cdbd2c0db2,0,93,1,0,0,0,0,0,0,1,...,0.0,0.0,34.0,1.0,36.0,0.0,13.583333,13.0,14.0,0.500000


,first_active_month,feature_3,first_active_delta_days,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,...,new_category_2_5.0_sum,new_category_3_A_sum,new_category_3_B_sum,new_category_3_C_sum,new_authorized_flag_y_sum,new_authorized_flag_n_sum,new_month_diff_mean,new_month_diff_min,new_month_diff_max,new_month_diff_std
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_0ab67a22ab,2017-04-01,1,307,0,0,1,0,0,0,0,...,0.0,0.0,2.0,1.0,3.0,0.0,15.333333,15.0,16.0,0.577350
C_ID_130fd0cbdd,2017-01-01,0,397,0,1,0,0,0,0,0,...,0.0,0.0,8.0,1.0,9.0,0.0,13.777778,13.0,14.0,0.440959
C_ID_b709037bc5,2017-08-01,1,185,0,0,0,0,1,1,0,...,1.0,0.0,1.0,1.0,2.0,0.0,14.000000,14.0,14.0,0.000000
C_ID_d27d835a9f,2017-12-01,0,63,0,1,0,0,0,1,0,...,1.0,0.0,5.0,5.0,10.0,0.0,13.600000,13.0,14.0,0.516398
C_ID_2b5e3df5c2,2015-12-01,1,794,0,0,0,0,1,1,0,...,0.0,0.0,4.0,1.0,6.0,0.0,14.000000,14.0,14.0,0.000000


card_id
C_ID_92a2005557   -0.820283
C_ID_3d0044924f    0.392913
C_ID_d639edf6cd    0.688056
C_ID_186d6a6901    0.142495
C_ID_cdbd2c0db2   -0.159749
Name: target, dtype: float64

In [4]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [7]:
train = reduce_mem_usage(train)

features = train.columns
categorical_feats = [c for c in features if 'feature_' in c]


# target = reduce_mem_usage(target)
# features = reduce_mem_usage(features)
# categorical_feats = reduce_mem_usage(categorical_feats)
# categorical_feats = categorical_feats.astype('bool')

# display(target)

Mem. usage decreased to 22.53 Mb (70.8% reduction)


In [8]:
def LGB_CV(
          max_depth,
          num_leaves,
          min_data_in_leaf,
          feature_fraction,
          bagging_fraction,
          lambda_l1
         ):
    
    ### Delete this
   
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    oof = np.zeros(train.shape[0])

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                               label=target.iloc[trn_idx],
                               categorical_feature=categorical_feats)
        val_data = lgb.Dataset(train.iloc[val_idx][features],
                               label=target.iloc[val_idx],
                               categorical_feature=categorical_feats)
    
        param = {
            'num_leaves': int(num_leaves),
            'min_data_in_leaf': int(min_data_in_leaf), 
            'objective':'regression',
            'max_depth': int(max_depth),
            'learning_rate': 0.01,
            "boosting": "gbdt",
            "feature_fraction": feature_fraction,
            "bagging_freq": 1,
            "bagging_fraction": bagging_fraction ,
            "bagging_seed": 11,
            "metric": 'rmse',
            "lambda_l1": lambda_l1,
            "verbosity": -1
        }
    
        clf = lgb.train(param,
                        trn_data,
                        10000,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=500,
                        early_stopping_rounds = 200)
        
        oof[val_idx] = clf.predict(train.iloc[val_idx][features],
                                   num_iteration=clf.best_iteration)
        
        del clf, trn_idx, val_idx
        gc.collect()
        
    return -mean_squared_error(oof, target)**0.5

In [9]:
from bayes_opt import BayesianOptimization
LGB_BO = BayesianOptimization(LGB_CV, {
    'max_depth': (4, 10),
    'num_leaves': (5, 130),
    'min_data_in_leaf': (10, 150),
    'feature_fraction': (0.7, 1.0),
    'bagging_fraction': (0.7, 1.0),
    'lambda_l1': (0, 6)
    })

In [10]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod(
            (datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))



In [12]:
import lightgbm as lgb
import gc
from sklearn.model_selection import KFold
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print('-'*126)

start_time = timer(None)
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=2, n_iter=20, acq='ei', xi=0.0)
timer(start_time)

------------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | baggin... | featur... | lambda_l1 | max_depth | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
fold n°0
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.60354	valid_1's rmse: 3.71716
[1000]	training's rmse: 3.54848	valid_1's rmse: 3.71279
Early stopping, best iteration is:
[1015]	training's rmse: 3.54743	valid_1's rmse: 3.71271
fold n°1
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.61964	valid_1's rmse: 3.6504
[1000]	training's rmse: 3.56864	valid_1's rmse: 3.64618
[1500]	training's rmse: 3.52758	valid_1's rmse: 3.64572
Early stopping, best iteration is:
[1409]	training's rmse: 3.53459	valid_1's rmse: 3.64541
fold n°2
Training until validation scores don't improve for 200 rounds.
[5

Early stopping, best iteration is:
[3954]	training's rmse: 3.63384	valid_1's rmse: 3.65236
|  4        | -3.698    |  0.8717   |  0.8906   |  0.4296   |  5.333    |  10.07    |  5.17     |
fold n°0
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.708	valid_1's rmse: 3.75415
[1000]	training's rmse: 3.68921	valid_1's rmse: 3.73887
[1500]	training's rmse: 3.67786	valid_1's rmse: 3.73203
[2000]	training's rmse: 3.66887	valid_1's rmse: 3.7277
[2500]	training's rmse: 3.66102	valid_1's rmse: 3.72485
[3000]	training's rmse: 3.65401	valid_1's rmse: 3.72267
[3500]	training's rmse: 3.64786	valid_1's rmse: 3.72152
[4000]	training's rmse: 3.64156	valid_1's rmse: 3.72049
[4500]	training's rmse: 3.63578	valid_1's rmse: 3.72002
[5000]	training's rmse: 3.6304	valid_1's rmse: 3.7195
Early stopping, best iteration is:
[4868]	training's rmse: 3.6318	valid_1's rmse: 3.71948
fold n°1
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 

fold n°3
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.51056	valid_1's rmse: 3.82093
Early stopping, best iteration is:
[656]	training's rmse: 3.49041	valid_1's rmse: 3.82036
fold n°4
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.55355	valid_1's rmse: 3.65258
Early stopping, best iteration is:
[571]	training's rmse: 3.54177	valid_1's rmse: 3.6522
|  8        | -3.694    |  0.9847   |  0.7683   |  0.7022   |  9.308    |  149.8    |  128.9    |
fold n°0
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.55617	valid_1's rmse: 3.71596
Early stopping, best iteration is:
[670]	training's rmse: 3.5285	valid_1's rmse: 3.71502
fold n°1
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.57311	valid_1's rmse: 3.64657
Early stopping, best iteration is:
[716]	training's rmse: 3.53898	valid_1's rmse: 3.64561
fold n°2
Training until validation s

Early stopping, best iteration is:
[624]	training's rmse: 3.50657	valid_1's rmse: 3.81972
fold n°4
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.56867	valid_1's rmse: 3.64889
Early stopping, best iteration is:
[480]	training's rmse: 3.57209	valid_1's rmse: 3.64877
|  14       | -3.692    |  0.7347   |  0.9704   |  1.028    |  9.668    |  149.9    |  129.5    |
fold n°0
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.69945	valid_1's rmse: 3.74658
[1000]	training's rmse: 3.67922	valid_1's rmse: 3.73161
[1500]	training's rmse: 3.66666	valid_1's rmse: 3.72558
[2000]	training's rmse: 3.6565	valid_1's rmse: 3.72214
[2500]	training's rmse: 3.6477	valid_1's rmse: 3.71986
[3000]	training's rmse: 3.63911	valid_1's rmse: 3.71821
[3500]	training's rmse: 3.63166	valid_1's rmse: 3.71758
[4000]	training's rmse: 3.62402	valid_1's rmse: 3.71695
Early stopping, best iteration is:
[4213]	training's rmse: 3.62117	valid_1's r

fold n°1
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.72572	valid_1's rmse: 3.68111
[1000]	training's rmse: 3.70683	valid_1's rmse: 3.66795
[1500]	training's rmse: 3.69557	valid_1's rmse: 3.66167
[2000]	training's rmse: 3.68676	valid_1's rmse: 3.65763
[2500]	training's rmse: 3.67912	valid_1's rmse: 3.65476
[3000]	training's rmse: 3.67225	valid_1's rmse: 3.65269
[3500]	training's rmse: 3.66536	valid_1's rmse: 3.65104
[4000]	training's rmse: 3.65948	valid_1's rmse: 3.65012
[4500]	training's rmse: 3.65391	valid_1's rmse: 3.64941
[5000]	training's rmse: 3.64853	valid_1's rmse: 3.64904
[5500]	training's rmse: 3.64296	valid_1's rmse: 3.64871
[6000]	training's rmse: 3.63803	valid_1's rmse: 3.64814
[6500]	training's rmse: 3.63285	valid_1's rmse: 3.64785
Early stopping, best iteration is:
[6638]	training's rmse: 3.63152	valid_1's rmse: 3.64774
fold n°2
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.7304	valid_1'

[2500]	training's rmse: 3.63025	valid_1's rmse: 3.64773
[3000]	training's rmse: 3.61896	valid_1's rmse: 3.64684
Early stopping, best iteration is:
[2982]	training's rmse: 3.61928	valid_1's rmse: 3.64678
fold n°2
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.69975	valid_1's rmse: 3.64192
[1000]	training's rmse: 3.67613	valid_1's rmse: 3.63244
[1500]	training's rmse: 3.65938	valid_1's rmse: 3.62956
[2000]	training's rmse: 3.64558	valid_1's rmse: 3.62875
[2500]	training's rmse: 3.63328	valid_1's rmse: 3.62804
Early stopping, best iteration is:
[2513]	training's rmse: 3.63292	valid_1's rmse: 3.62793
fold n°3
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.65096	valid_1's rmse: 3.83467
[1000]	training's rmse: 3.62712	valid_1's rmse: 3.82619
[1500]	training's rmse: 3.61089	valid_1's rmse: 3.8228
[2000]	training's rmse: 3.59732	valid_1's rmse: 3.82162
[2500]	training's rmse: 3.58542	valid_1's rmse: 3.82112
[3000